# SPDE

In this tutorial, we show how the use of SPDE for Varying Anisotropy in the Simulation process

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

Defining some global parameters

In [ ]:
#Extension of the simulation grid
nxmax = 500 
nymax = 200

#Well Definition
nwell = 6
pas   = 3

#Anisotropy ratio 
ratio=1.5
range=150

#Some seeds
seed1  = 34556643
seed2  = 244212
seednc = 432432
seedw  = 2432145

# Color Scale
zlim = [-1.6, 2.5]

nostatKeys = ["A","R2"]

Internal function

In [ ]:
def make_well(res,nwell=nwell,pas=pas,seed=seedw):
  set.seed(seedw)
  nxmax = res.nx[2]
  indexes=unique(sample(nxmax,nwell))
  x1=NULL
  x2=NULL
  for i in indexes:
    temp=range(ceiling(db[i,3]), floor(db[i,4]))
    temp=temp[1:floor(length(temp)/pas)*pas]
    x1=c(x1,rep(i,length(temp))) + 0.2
    x2=c(x2,temp)

  res = db.locate(res,4,"sel")
  db.sample = db.create(x1=x1,x2=x2)
  db.sample = migrate(res,db.sample,7)
  db.sample

Simulating the layer boundaries

In [ ]:
db = gl.DbGrid.create(nx=nxmax)
model = gl.Model.createFromParam(gl.ECov.GAUSSIAN,range=200,space=gl.SpaceRN(1))
err = gl.simtub(None,dbout=db,model=model,nbtuba=1000,seed=seed1,namconv=gl.NamingConvention("W1"))
err = gl.simtub(None,dbout=db,model=model,nbtuba=1000,seed=seed2,namconv=gl.NamingConvention("W2"))
db["W1"]=db["W1"]-min(db["W1"])
db["W2"]=db["W2"]-min(db["W2"])
db["W2"]=db["W1"]+db["W2"]+1
db["W1"]=nymax*db["W1"]/max(db["W2"])
db["W2"]=nymax*db["W2"]/max(db["W2"])

Plotting the limits of the layer

In [ ]:
ax = gp.grid1D(db,"W1", color="blue", title="Layer limits")
ax = gp.grid1D(db,"W2", color="green", ax=ax)

Creation of the varying anisotropy ("directed" by the two layers)

In [ ]:
model = gl.Model.createFromParam(gl.ECov.BESSEL_K,range=range,param=1,space=gl.SpaceRN(2))
dbsim = gl.DbGrid.create([nxmax,nymax])
ind = (dbsim["x1"]).reshape(1,-1)[0].astype(int)
dbsim["sel"] = (dbsim["x2"] > db[ind,"W1"]) & (dbsim["x2"] < db[ind,"W2"])
anglesi = np.arctan(db["W1"][1:]-db["W1"][:-1])/np.pi*180
angless = np.arctan(db["W2"][1:]-db["W2"][:-1])/np.pi*180
anglesi = np.insert(anglesi, 0, anglesi[0])
angless = np.insert(angless, 0, angless[0])

aniso = np.divide(dbsim["x2"]-db[ind,"W1"], db[ind,"W2"]-db[ind,"W1"])
aniso = anglesi[ind] + np.multiply(aniso[0], angless[ind]-anglesi[ind])
ratio = ratio*(db[ind,"W2"]-db[ind,"W1"])/max(db["W2"]-db["W1"])

dbsim.addColumns(aniso,"aniso")
dbsim.addColumns(ratio,"ratio")
dbsim.setLocator("sel",gl.ELoc.SEL)
dbsim.setLocators(["aniso","ratio"],gl.ELoc.NOSTAT)

In [ ]:
ax = gp.grid(dbsim,name="aniso",title="Anisotropy Angle")
ax = gp.grid(dbsim,name="ratio",title="Anisotropy Ratio")

Display the anisotropy maps (on a coarser grid)

In [ ]:
#display.model(dbsim,model)
#display.layer(db)

Non-conditional simulations

In [ ]:
mesh = gl.MeshETurbo(dbsim)
nostat = gl.NoStatArray(nostatKeys, dbsim)
err = model.addNoStat(nostat)

In [ ]:
spde = gl.SPDE()
spde.init(model,dbsim,None,gl.ESPDECalcMode.SIMUNONCOND, mesh)
spde.compute()
iuid = spde.query(dbsim)

In [ ]:
ax = gp.grid(dbsim)

In [ ]:
#spdeRes = gl.SPDE()
#spdeRes.init(model,grid,dat,gl.ESPDECalcMode.KRIGING,mesh)
#spdeRes.compute()